# Fooooooooooooooocus
Colab NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Colab](https://colab.research.google.com) 部署 Fooocus 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式，并安装 [Fooocus](https://github.com/lllyasviel/Fooocus)。
2. 下载模型：下载可选列表中的模型（可选）。
3. 启动 Fooocus：启动 Fooocus，并显示访问地址。
4. 其他功能：包含自定义模型下载工具和调整内网穿透参数的工具（可选）。

## 使用
1. 在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择`GPU T4`或者其他 GPU。
2. 运行`环境配置`单元。
3. 运行`启动 Fooocus（图片保存在 Google Drive）`单元，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。

## 提示
1. Colab 在关机后将会清除所有数据，所以每次重新启动时必须运行`环境配置`单元再运行`启动 Fooocus（图片保存在 Google Drive）`单元。
2. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
3. 每次启动 Colab 后必须运行`环境配置`单元，才能运行`启动 Fooocus`单元启动 Fooocus。
4. 其他功能有自定义模型下载等功能，根据自己的需求进行使用。
5. 运行`启动 Fooocus（图片保存在 Google Drive）`时将弹出 Google Drive 访问授权提示，根据提示进行授权。授权后，使用 Fooocus 生成的图片将保存在 Google Drive 的`fooocus_output`文件夹中。
6. Gradio 的内网穿透地址可在启动 Fooocus 后日志中的`Running on public URL`查看。

In [ ]:
#@title 👇 环境配置
INIT_CONFIG = 1


def echo(*args):
    """格式化消息输出"""
    import time
    t = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    for i in args:
        print(f"[{t}]:: {i}")



class ARIA2:
    """基于 Aria2 的文件下载工具"""
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def aria2(self, url, path, filename):
        """调用 Aria2 下载文件

        参数:
            url:
                文件的下载链接

            path:
                将文件下载到本地的路径

            filename:
                将要下载的文件重命名

        返回值:
            `str`: 文件保存路径
        """
        import os
        file_path = os.path.join(path, filename)
        if not os.path.exists(file_path):
            echo(f"开始下载 {filename} ，路径: {file_path}")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{path}" -o "{filename}"
            if os.path.exists(file_path) and not os.path.exists(file_path + ".aria2"):
                echo(f"{filename} 下载完成")
                return file_path
            else:
                echo(f"{filename} 下载中断")
                return None
        else:
            if os.path.exists(file_path + ".aria2"):
                echo(f"开始下载 {filename} ，路径: {path}/{filename}")
                !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{path}" -o "{filename}"
                if os.path.exists(file_path) and not os.path.exists(file_path + ".aria2"):
                    echo(f"{filename} 下载完成")
                    return file_path
                else:
                    echo(f"{filename} 下载中断")
                    return None
            else:
                echo(f"{filename} 文件已存在，路径: {file_path}")
                return file_path



# GIT
class GIT:
    """基于 Git 命令的模块"""
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def exists(self, addr=None, path=None, name=None):
        """检测要克隆的项目是否存在于指定路径

        参数:
            addr:
                Git 仓库的地址

            path:
                将 Git 仓库下载到本地的路径

            name:
                将 Git 仓库进行重命名
        """
        import os
        if addr is not None:
            if path is None and name is None:
                path = os.path.join(os.getcwd(), addr.split("/").pop().split(".git", 1)[0])
            elif path is None and name is not None:
                path = os.path.join(os.getcwd(), name)
            elif path is not None and name is None:
                path = os.path.join(os.path.normpath(path), addr.split("/").pop().split(".git", 1)[0])
            elif path is not None and name is not None:
                path = os.path.join(os.path.normpath(path), name)

        if os.path.exists(path):
            return True
        else:
            return False


    def clone(self, addr, path=None, name=None):
        """克隆 Git 仓库到本地

        参数:
            addr:
                Git 仓库的地址

            path:
                将 Git 仓库下载到本地的路径

            name:
                将 Git 仓库进行重命名

        返回值:
            `str`: 下载的仓库地址
        """
        import os
        repo = addr.split("/").pop().split(".git", 1)[0]
        if path is None and name is None:
            path = os.getcwd()
            name = repo
        elif path is not None and name is None:
            name = repo
        elif path is None and name is not None:
            path = os.getcwd()

        repo_path = os.path.join(path, name)

        if not self.exists(addr, path, name):
            echo(f"开始下载 {repo}")
            !git clone {addr} "{repo_path}" --recurse-submodules
            if os.path.exists(repo_path):
                echo(f"{repo} 下载成功, 路径: {repo_path}")
                return repo_path
            else:
                echo(f"{repo} 下载失败")
                return None
        else:
            echo(f"{repo} 已存在, 路径: {repo_path}")
            return repo_path



class TUNNEL:
    """内网穿透工具"""
    LOCALHOST_RUN = "localhost.run"
    REMOTE_MOE = "remote.moe"
    WORKSPACE = ""
    WORKFOLDER = ""
    PORT = ""


    def __init__(self, workspace, workfolder, port) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder
        self.PORT = port


    def ngrok(self, ngrok_token: str) -> str:
        """使用 Ngrok 内网穿透

        参数:
            ngrok_token (`str`):
                Ngrok 账号 Token

        返回值:
            `str`:
                Ngrok 内网穿透生成的访问地址
        """
        from pyngrok import conf, ngrok
        conf.get_default().auth_token = ngrok_token
        conf.get_default().monitor_thread = False
        port = self.PORT
        ssh_tunnels = ngrok.get_tunnels(conf.get_default())
        if len(ssh_tunnels) == 0:
            ssh_tunnel = ngrok.connect(port, bind_tls=True)
            return ssh_tunnel.public_url
        else:
            return ssh_tunnels[0].public_url


    def cloudflare(self) -> str:
        """使用 CloudFlare 内网穿透

        返回值:
            `str`:
                CloudFlare 内网穿透生成的访问地址
        """
        from pycloudflared import try_cloudflare
        port = self.PORT
        urls = try_cloudflare(port).tunnel
        return urls


    from typing import Union
    from pathlib import Path


    def gen_key(self, path: Union[str, Path]) -> None:
        """生成 SSH 密钥

        参数:
            path (`str` | `Path`):
                生成 SSH 密钥的路径
        """
        import subprocess
        import shlex
        from pathlib import Path
        path = Path(path)
        arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
        args = shlex.split(arg_string)
        subprocess.run(args, check=True)
        path.chmod(0o600)


    def ssh_tunnel(self, host: str) -> None:
        """使用 SSH 进行内网穿透

        参数:
            host (`str`):
                提供 SSH 内网穿透的服务器地址

        返回值:
            (`str` | `None`):
                使用内网穿透得到的访问地址, 如果启动内网穿透失败则不返回地址
        """
        import subprocess
        import atexit
        import shlex
        import re
        import os
        from pathlib import Path
        from tempfile import TemporaryDirectory

        ssh_name = "id_rsa"
        ssh_path = Path(self.WORKSPACE) / ssh_name
        port = self.PORT

        tmp = None
        if not ssh_path.exists():
            try:
                self.gen_key(ssh_path)
            # write permission error or etc
            except subprocess.CalledProcessError:
                tmp = TemporaryDirectory()
                ssh_path = Path(tmp.name) / ssh_name
                self.gen_key(ssh_path)

        arg_string = f"ssh -R 80:127.0.0.1:{port} -o StrictHostKeyChecking=no -i {ssh_path.as_posix()} {host}"
        args = shlex.split(arg_string)

        tunnel = subprocess.Popen(
            args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)
        if tmp is not None:
            atexit.register(tmp.cleanup)

        tunnel_url = ""
        LOCALHOST_RUN = self.LOCALHOST_RUN
        lines = 27 if host == LOCALHOST_RUN else 5
        localhostrun_pattern = re.compile(r"(?P<url>https?://\S+\.lhr\.life)")
        remotemoe_pattern = re.compile(r"(?P<url>https?://\S+\.remote\.moe)")
        pattern = localhostrun_pattern if host == LOCALHOST_RUN else remotemoe_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")

            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                if lines == 27:
                    os.environ['LOCALHOST_RUN'] = tunnel_url
                    return tunnel_url
                else:
                    os.environ['REMOTE_MOE'] = tunnel_url
                    return tunnel_url
                # break
        else:
            echo(f"启动 {host} 内网穿透失败")


    def localhost_run(self):
        """使用 localhost.run 进行内网穿透"""
        urls = self.ssh_tunnel(self.LOCALHOST_RUN)
        return urls


    def remote_moe(self):
        """使用 remote.moe 进行内网穿透"""
        urls = self.ssh_tunnel(self.REMOTE_MOE)
        return urls


    def gradio(self):
        """使用 Gradio 进行内网穿透"""
        import subprocess
        import shlex
        import atexit
        import re
        port = self.PORT
        cmd = f"gradio-tunneling --port {port}"
        cmd = shlex.split(cmd)
        tunnel = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)

        tunnel_url = ""
        lines = 5
        gradio_pattern = re.compile(r"(?P<url>https?://\S+\.gradio\.live)")
        pattern = gradio_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")
            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                return tunnel_url
        else:
            echo(f"启动 Gradio 内网穿透失败")


    def start(
            self,
            ngrok: bool = False,
            ngrok_token: str = None,
            cloudflare: bool = False,
            remote_moe: bool = False,
            localhost_run: bool = False,
            gradio: bool = False
    ):
        """启动内网穿透

        参数:
            ngrok (`bool`):
                启用 Ngrok 内网穿透

            ngrok_token (`str`):
                Ngrok 账号 Token

            cloudflare (`bool`):
                启用 CloudFlare 内网穿透

            remote_moe (`bool`):
                启用 remote.moe 内网穿透

            localhost_run (`bool`):
                使用 localhost.run 内网穿透

            gradio (`bool`):
                使用 Gradio 内网穿透
        """
        if cloudflare is True or ngrok is True or ngrok_token is not None or remote_moe is True or localhost_run is True or gradio is True:
            echo("启动内网穿透")

        if cloudflare is True:
            cloudflare_url = self.cloudflare()
        else:
            cloudflare_url = None

        if ngrok is True and ngrok_token is not None:
            ngrok_url = self.ngrok(ngrok_token)
        else:
            ngrok_url = None

        if remote_moe is True:
            remote_moe_url = self.remote_moe()
        else:
            remote_moe_url = None

        if localhost_run is True:
            localhost_run_url = self.localhost_run()
        else:
            localhost_run_url = None

        if gradio is True:
            gradio_url = self.gradio()
        else:
            gradio_url = None

        echo("下方为访问地址")
        print("==================================================================================")
        echo(f"CloudFlare: {cloudflare_url}")
        echo(f"Ngrok: {ngrok_url}")
        echo(f"remote.moe: {remote_moe_url}")
        echo(f"localhost_run: {localhost_run_url}")
        echo(f"Gradio: {gradio_url}")
        print("==================================================================================")



class ENV:
    """提供初始化环境的功能"""
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def prepare_env_depend(self, use_uv: bool = False):
        """为 Notebook 的功能准备必须的环境依赖

        参数:
            use_uv (bool):
                是否使用 uv 代替 Pip 安装 Python 软件包
        """
        pip_mirror = "--index-url https://pypi.python.org/simple --extra-index-url https://download.pytorch.org/whl/cu121"

        echo("安装自身组件依赖")
        !pip install uv {pip_mirror}
        pkg = "huggingface_hub modelscope"
        if use_uv:
            !uv pip install {pkg} {pip_mirror} --system --quiet -U || pip install {pkg} {pip_mirror} -U
        else:
            !pip install {pkg} {pip_mirror} -U
        !apt update
        !apt install aria2 google-perftools p7zip-full unzip tree -y


    def prepare_torch(self, torch_ver, xformers_ver, use_uv: bool = False):
        """安装 PyTorch 和 xFormers

        参数:
            torch_ver:
                PyTorch 软件包名称和版本信息, 如`torch==2.0.0 torchvision==0.15.1`

            xformers_ver:
                xFormers 软件包名称和版本信息, 如`xformers==0.0.18`

            use_uv (bool):
                使用 uv 代替 Pip 进行 Python 软件包安装
        """
        pip_mirror = "--index-url https://download.pytorch.org/whl/cu121"
        if use_uv:
            if torch_ver != "":
                echo("安装 PyTorch")
                !uv pip install {torch_ver} {pip_mirror} --system --quiet || pip install {torch_ver} {pip_mirror}
            if xformers_ver != "":
                echo("安装 xFormers")
                !uv pip install {xformers_ver} {pip_mirror} --no-deps --system --quiet || pip install {xformers_ver} {pip_mirror} --no-deps
        else:
            if torch_ver != "":
                echo("安装 PyTorch")
                !pip install {torch_ver} {pip_mirror}
            if xformers_ver != "":
                echo("安装 xFormers")
                !pip install {xformers_ver} {pip_mirror} --no-deps


    def install_requirements(self, path, use_uv: bool = False):
        """从文件 (requirements.txt) 中安装 Python 软件包

        参数:
            path:
                依赖记录文件的路径

            use_uv (bool):
                使用 uv 代替 Pip 进行 Python 软件包安装
        """
        import os
        pip_mirror = "--index-url https://pypi.python.org/simple --extra-index-url https://download.pytorch.org/whl/cu121"
        if os.path.exists(path):
            echo("安装依赖")
            if use_uv:
                !uv pip install -r "{path}" {pip_mirror} --system --quiet || pip install -r "{path}" {pip_mirror}
            else:
                !pip install -r "{path}" {pip_mirror}
        else:
            echo("依赖文件路径为空")


    def tcmalloc_colab(self):
        """配置 TCMalloc 内存优化"""
        echo("配置内存优化")
        import os
        aria2 = ARIA2(self.WORKSPACE, self.WORKFOLDER)
        path = self.WORKSPACE
        libtcmalloc_path = os.path.join(self.WORKSPACE, "libtcmalloc_minimal.so.4")
        aria2.aria2("https://github.com/licyk/term-sd/releases/download/archive/libtcmalloc_minimal.so.4", path, "libtcmalloc_minimal.so.4")
        os.environ["LD_PRELOAD"] = libtcmalloc_path



class MANAGER:
    """环境管理"""
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def clear_up(self):
        """清理 Notebook 的内容输出"""
        from IPython.display import clear_output
        clear_output(wait=False)


    def check_gpu(self):
        """检查环境中是否有可用的 GPU"""
        echo("检测 GPU 是否可用")
        import tensorflow as tf
        echo(f"TensorFlow 版本: {tf.__version__}")
        if tf.test.gpu_device_name():
            echo("GPU 可用")
        else:
            echo("GPU 不可用")
            raise Exception("\n没有可用的 GPU, 请在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择 GPU T4\n如果不能使用 GPU, 请尝试更换账号!")


    def config_google_drive(self):
        """配置 Google Drive"""
        echo("挂载 Google Drive 中, 请根据提示进行操作")
        import os
        if not os.path.exists('/content/drive/MyDrive'):
            from google.colab import drive
            try:
                drive.mount('/content/drive')
            except Exception as e:
                raise Exception("挂载 Google Drive 时出现问题", e)
            echo("Google Dirve 挂载完成")
        else:
            echo("Google Drive 已挂载")

        # 检测并创建输出文件夹
        if os.path.exists('/content/drive/MyDrive'):
            if not os.path.exists('/content/drive/MyDrive/fooocus_output'):
                echo("在 Google Drive 创建 fooocus_ouput 文件夹")
                !mkdir -p /content/drive/MyDrive/fooocus_output
        else:
            raise Exception("未挂载 Google Drive, 请重新挂载后重试")



class ModelDownload:
    """模型下载器"""
    def __init__(self, urls) -> None:
        import threading
        from queue import Queue
        self.urls = urls
        self.queue = Queue()
        self.aria2 = ARIA2(None, None)
        self.total_urls = len(urls)  # 记录总的URL数
        self.downloaded_count = 0  # 记录已下载的数量
        self.lock = threading.Lock()  # 创建锁以保护对下载计数器的访问


    def worker(self):
        """调用文件下载工具"""
        while True:
            url = self.queue.get()
            if url is None:
                break
            self.aria2.aria2(url[0], url[1], url[2])
            self.queue.task_done()
            with self.lock:  # 访问共享资源时加锁
                self.downloaded_count += 1
                self.print_progress()  # 打印进度


    def print_progress(self):
        """进度条显示"""
        progress = (self.downloaded_count / self.total_urls) * 100
        echo(f"模型下载进度: {self.downloaded_count}/{self.total_urls} ({progress:.2f}%)")


    def start_threads(self, num_threads: int = 16):
        """启动多线程下载器

        参数:
            num_threads (`int`):
                下载线程
        """
        import threading
        threads = []
        for _ in range(num_threads):
            thread = threading.Thread(target=self.worker)
            thread.start()
            threads.append(thread)

        for url in self.urls:
            self.queue.put(url)

        self.queue.join()

        for _ in range(num_threads):
            self.queue.put(None)

        for thread in threads:
            thread.join()



class FOOOCUS(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    """Fooocus 管理工具"""
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 7865)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def get_sd_model(self, url, filename = None):
        """下载大模型"""
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/checkpoints"
        filename = url.split("/").pop() if filename is None else filename
        super().aria2(url, path, filename)


    def get_lora_model(self, url, filename = None):
        """下载 LoRA 模型"""
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/loras"
        filename = url.split("/").pop() if filename is None else filename
        super().aria2(url, path, filename)


    def install_config(self):
        """下载 Fooocus 配置文件"""
        path = self.WORKSPACE + "/" + self.WORKFOLDER
        echo("下载配置文件")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json", path + "/presets", "custom.json")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json", path, "config.txt")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json", path + "/language", "zh.json")


    def pre_download_model(self, path):
        """根据 Fooocus 配置文件预下载模型"""
        import os
        import json

        if os.path.exists(path):
            try:
                with open(path, "r", encoding="utf8") as file:
                    data = json.load(file)
                    echo("预下载 Fooocus 模型中")
            except Exception:
                # json 文件格式出现问题
                data = {}
        else:
            data = {}

        sd_model_list = data.get("checkpoint_downloads") if not isinstance(data.get("checkpoint_downloads"), type(None)) else {}
        lora_list = data.get("lora_downloads") if not isinstance(data.get("lora_downloads"), type(None)) else {}
        vae_list = data.get("vae_downloads") if not isinstance(data.get("vae_downloads"), type(None)) else {}
        sd_model_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "checkpoints")
        lora_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "loras")
        vae_path = os.path.join(self.WORKSPACE, self.WORKFOLDER, "models", "vae")

        model_url = []
        for i in sd_model_list:
            model_url.append([sd_model_list.get(i), sd_model_path, i])

        for i in lora_list:
            model_url.append([lora_list.get(i), lora_path, i])

        for i in vae_list:
            model_url.append([vae_list.get(i), vae_path, i])

        model_downloader = ModelDownload(model_url)
        model_downloader.start_threads()

        echo("预下载 Fooocus 模型完成")


    def install(self, torch_ver, xformers_ver, use_uv):
        """安装 Fooocus
        
        参数:
            torch_ver:
                指定的 PyTorch 软件包包名, 并包括版本号

            xformers_ver:
                指定的 xFormers 软件包包名, 并包括版本号

            use_uv:
                使用 uv 替代 Pip 进行 Python 软件包的安装
        """
        import os
        os.chdir(self.WORKSPACE)
        req_file = os.path.join(self.WORKSPACE, self.WORKFOLDER, "requirements_versions.txt")
        config_file = os.path.join(self.WORKSPACE, self.WORKFOLDER, "presets", "custom.json")
        self.check_gpu()
        self.prepare_env_depend(use_uv)
        self.clone("https://github.com/lllyasviel/Fooocus", self.WORKSPACE, self.WORKFOLDER)
        os.chdir(os.path.join(self.WORKSPACE, self.WORKFOLDER))
        self.prepare_torch(torch_ver, xformers_ver, use_uv)
        self.install_requirements(req_file, use_uv)
        if use_uv: # 纠正 PyTorch 版本
            self.prepare_torch(torch_ver, xformers_ver, use_uv)

        self.install_config()
        self.tcmalloc_colab()
        self.pre_download_model(config_file)


#######################################################


#@markdown ### 核心组件版本：
#@markdown #### PyTorch：
pytorch_ver = "torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121"  #@param {type:"string"}
#@markdown #### xFormers：
xformers_ver = "xformers==0.0.27"  #@param {type:"string"}
#@markdown ---
#@markdown - 使用 uv 作为 Python 包管理器
use_uv = True #@param {type:"boolean"}
#@markdown ---
#@markdown ### 选择内网穿透的方式（对应下方的勾选框）：
#@markdown - 使用 remote.moe 内网穿透
#@markdown - 使用 localhost.run 内网穿透
#@markdown - 使用 CloudFlare 内网穿透
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
#@markdown - 使用 Gradio 内网穿透
use_remote_moe = True #@param {type:"boolean"}
use_localhost_run = True #@param {type:"boolean"}
use_cloudflare = True #@param {type:"boolean"}
use_ngrok = False #@param {type:"boolean"}
use_gradio = True #@param {type:"boolean"}
#@markdown ### Ngrok Token：
ngrok_token = ""  #@param {type:"string"}


#######################################################


fooocus = FOOOCUS("/content","Fooocus")
fooocus.install(pytorch_ver, xformers_ver, use_uv)
fooocus.clear_up(False)
echo("Fooocus 运行环境配置完成")

In [ ]:
#@title 👇 下载模型（可选）

try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

# 模型下载
echo("下载模型中")
#@markdown 选择下载的模型：
##############################


sd_xl_base = False  #@param {type:"boolean"}
if sd_xl_base:
    fooocus.get_sd_model("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors")

anima_pencil_xl = False  #@param {type:"boolean"}
if anima_pencil_xl:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323674", "anima_pencil_xl.safetensors")

bluePencilXL  = False  #@param {type:"boolean"}
if bluePencilXL:
    fooocus.get_sd_model("https://civitai.com/api/download/models/323375", "bluePencilXL_v050.safetensors")

AnythingXL_xl = False  #@param {type:"boolean"}
if AnythingXL_xl:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/AnythingXL_xl.safetensors")

abyssorangeXLElse_v10 = False  #@param {type:"boolean"}
if abyssorangeXLElse_v10:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/abyssorangeXLElse_v10.safetensors")

CounterfeitXL  = False  #@param {type:"boolean"}
if CounterfeitXL:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors")

animeIllustDiffusion  = False  #@param {type:"boolean"}
if animeIllustDiffusion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors")

nekorayxl = False  #@param {type:"boolean"}
if nekorayxl:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors")

animagine_XL_3 = False  #@param {type:"boolean"}
if animagine_XL_3:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors")

animagine_XL_3_1 = False  #@param {type:"boolean"}
if animagine_XL_3_1:
    fooocus.get_sd_model("https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors")

heartOfAppleXL_v20 = False  #@param {type:"boolean"}
if heartOfAppleXL_v20:
    fooocus.get_sd_model("https://civitai.com/api/download/models/337306", "heartOfAppleXL_v30.safetensors")

heartOfAppleXL_v30 = False  #@param {type:"boolean"}
if heartOfAppleXL_v30:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors")

holodayo_xl_21 = False  #@param {type:"boolean"}
if holodayo_xl_21:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors")

kivotos_xl_20 = False  #@param {type:"boolean"}
if kivotos_xl_20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors")

sd_xl_anime_V52 = False  #@param {type:"boolean"}
if sd_xl_anime_V52:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_anime_V52.safetensors")

kohakuXLGamma_rev1  = False  #@param {type:"boolean"}
if kohakuXLGamma_rev1:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-gamma/resolve/main/kohaku-xl-gamma-rev1.safetensors")

kohakuXLBeta_beta7  = False  #@param {type:"boolean"}
if kohakuXLBeta_beta7:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLBeta_beta7.safetensors")

Kohaku_XL_Delta = False  #@param {type:"boolean"}
if Kohaku_XL_Delta:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Delta/resolve/main/kohaku-xl-delta-rev1.safetensors")

kohakuXL_Epsilon = False  #@param {type:"boolean"}
if kohakuXL_Epsilon:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon/resolve/main/kohaku-xl-epsilon-rev1.safetensors")

kohakuXL_Epsilon_rev2 = False  #@param {type:"boolean"}
if kohakuXL_Epsilon_rev2:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev2/resolve/main/kohaku-xl-epsilon-rev2.safetensors")

kohakuXL_Epsilon_rev3 = False  #@param {type:"boolean"}
if kohakuXL_Epsilon_rev3:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev3/resolve/main/kohaku-xl-epsilon-rev3.safetensors")

kohakuXL_Zeta = True  #@param {type:"boolean"}
if kohakuXL_Zeta:
    fooocus.get_sd_model("https://huggingface.co/KBlueLeaf/Kohaku-XL-Zeta/resolve/main/kohaku-xl-zeta.safetensors")

starryXLV52 = False  #@param {type:"boolean"}
if starryXLV52:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors")

BArtstyleDB = False  #@param {type:"boolean"}
if BArtstyleDB:
    fooocus.get_sd_model("https://civitai.com/api/download/models/299095", "BArtstyleDB_XL.safetensors")

BArtstyleDB_3 = False  #@param {type:"boolean"}
if BArtstyleDB_3:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBlueArchiveFlatCelluloidStyle_xlv3.safetensors")

pony_v6 = False  #@param {type:"boolean"}
if pony_v6:
    fooocus.get_sd_model("https://civitai.com/api/download/models/290640", "ponyDiffusionV6XL_v6StartWithThisOne.safetensors")

pdForAnime_v20 = False  #@param {type:"boolean"}
if pdForAnime_v20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors")

tPonynai3_v51 = False  #@param {type:"boolean"}
if tPonynai3_v51:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tPonynai3_v51WeightOptimized.safetensors")

omegaPonyXLAnime_v20 = False  #@param {type:"boolean"}
if omegaPonyXLAnime_v20:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors")

Illustrious_XL_v01 = True  #@param {type:"boolean"}
if Illustrious_XL_v01:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors")

noobaiXLNAIXL_earlyAccessVersion = False  #@param {type:"boolean"}
if noobaiXLNAIXL_earlyAccessVersion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors")

noobaiXLNAIXL_epsilonPred05Version = False  #@param {type:"boolean"}
if noobaiXLNAIXL_epsilonPred05Version:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors")

noobaiXLNAIXL_epsilonPred075 = False  #@param {type:"boolean"}
if noobaiXLNAIXL_epsilonPred075:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors")

noobaiXLNAIXL_epsilonPred077 = False  #@param {type:"boolean"}
if noobaiXLNAIXL_epsilonPred077:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors")

noobaiXLNAIXL_epsilonPred10Version = False  #@param {type:"boolean"}
if noobaiXLNAIXL_epsilonPred10Version:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors")

noobaiXLNAIXL_epsilonPred11Version = True  #@param {type:"boolean"}
if noobaiXLNAIXL_epsilonPred11Version:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors")

noobaiXLNAIXL_vPredTestVersion = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPredTestVersion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors")

noobaiXLNAIXL_vPred05Version = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPred05Version:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors")

noobaiXLNAIXL_vPred06Version = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPred06Version:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors")

noobaiXLNAIXL_vPred065SVersion = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPred065SVersion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors")

noobaiXLNAIXL_vPred075SVersion = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPred075SVersion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors")

noobaiXLNAIXL_vPred09RVersion = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPred09RVersion:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors")

noobaiXLNAIXL_vPred10Version = False  #@param {type:"boolean"}
if noobaiXLNAIXL_vPred10Version:
    fooocus.get_sd_model("https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors")


##############################
fooocus.clear_up(False)
echo("模型下载完成")

In [ ]:
#@title 👇 启动 Fooocus（图片保存在 Google Drive）
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

echo("启动 Fooocus")
import os
os.chdir("/content/Fooocus")
fooocus.config_google_drive()
fooocus.tun.start(ngrok=use_ngrok, ngrok_token=ngrok_token, cloudflare=use_cloudflare, remote_moe=use_remote_moe, localhost_run=use_localhost_run, gradio=False)
if use_gradio:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram --share
else:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram
fooocus.clear_up(False)
echo("已关闭 Fooocus")

### ✨ 其他功能

In [ ]:
#@title 👇 自定义模型下载
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

#@markdown ### 选择模型种类：
#@markdown - Stable Diffusion（大模型）模型
#@markdown - LoRA模型
model_type = "LoRA" # @param ["Stable Diffusion", "LoRA"]
#@markdown ###填写模型的下载链接：
model_url = "https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/CoolFlatColor.safetensors"  #@param {type:"string"}
#@markdown ###填写模型的名称（包括后缀名）：
model_name = "CoolFlatColor.safetensors"  #@param {type:"string"}

if model_type == "Stable Diffusion":
    fooocus.get_sd_model(
        url=model_url,
        filename=model_name if model_name != "" else model_url.split("/").pop()
    )
elif model_type == "LoRA":
    fooocus.get_lora_model(
        url=model_url,
        filename=model_name if model_name != "" else model_url.split("/").pop()
    )

In [ ]:
#@title 👇 启动Fooocus（图片保存在 Colab）
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")
echo("启动 Fooocus")
import os
os.chdir("/content/Fooocus")
fooocus.tun.start(ngrok=use_ngrok, ngrok_token=ngrok_token, cloudflare=use_cloudflare, remote_moe=use_remote_moe, localhost_run=use_localhost_run, gradio=False)
if use_gradio:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram --share
else:
    !python /content/Fooocus/launch.py --preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram
fooocus.clear_up(False)
echo("已关闭 Fooocus")

In [ ]:
#@title 👇挂载 Google Drive 并备份图片
try:    
    i = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")
fooocus.config_google_drive()
echo("备份图片中")
!cp -r /content/drive/MyDrive/fooocus_output/* /content/drive_/MyDrive/fooocus_output
echo("备份完成")

In [ ]:
#@title 👇远程共享方式修改
#@markdown ### 选择内网穿透的方式（对应下方的勾选框）：
#@markdown - 使用 remote.moe 内网穿透
#@markdown - 使用 localhost.run 内网穿透
#@markdown - 使用 CloudFlare 内网穿透
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
#@markdown - 使用 Gradio 内网穿透
use_remote_moe = True #@param {type:"boolean"}
use_localhost_run = True #@param {type:"boolean"}
use_cloudflare = True #@param {type:"boolean"}
use_ngrok = False #@param {type:"boolean"}
use_gradio = True #@param {type:"boolean"}
#@markdown ### Ngrok Token：
ngrok_token = ""  #@param {type:"string"}

echo("设置共享方式完成")